
<a id='testing'></a>
How to read this lecture…

- If using QuantEcon lectures for the first time on a computer, execute ] add InstantiateFromURL inside of a notebook or the REPL  
- For some notebooks, enable content with “Trust” on the command tab of Jupyter lab  
- Code should execute sequentially if run in a Jupyter notebook  
- Please direct feedback to [contact@quantecon.org](mailto:contact@quantecon.org") or [discourse forum](http://discourse.quantecon.org/)  

# Packages, Testing, and Continuous Integration

Co-authored with Arnav Sood

## Contents

- [Packages, Testing, and Continuous Integration](#Packages,-Testing,-and-Continuous-Integration)  
  - [Project Setup](#Project-Setup)  
  - [Project Structure](#Project-Structure)  
  - [Project Workflow](#Project-Workflow)  
  - [Unit Testing](#Unit-Testing)  
  - [Continuous Integration with Travis](#Continuous-Integration-with-Travis)  
  - [CodeCoverage](#CodeCoverage)  
  - [Benchmarking](#Benchmarking)  
  - [Additional Notes](#Additional-Notes)  

This lecture is about structuring your project as a Julia module, and testing it with tools from GitHub

Benefits include

- Specifying dependencies (and their versions), so that your project works across Julia setups and over time  
- Being able to load your project’s functions from outside without copy/pasting  
- Writing tests that run locally, *and automatically on the GitHub server*  
- Having GitHub test your project across operating systems, Julia versions, etc.  

## Project Setup

### Account Setup

#### Travis CI

As we’ll see later, Travis is a service that automatically tests your project on the GitHub server

First, we need to make sure that your GitHub account is set up with Travis CI and CodeCov

As a reminder, make sure you signed up for the GitHub [Student Developer Pack](https://education.github.com/pack/) or Academic Plan <https://help.github.com/articles/applying-for-an-academic-research-discount/>_ if eligible

Navigate to the [Travis website](https://travis-ci.com/) and click “sign up with GitHub.” Supply your credentials

If you get stuck, see the [Travis tutorial](https://docs.travis-ci.com/user/tutorial/)

**NOTE** As of May 2018, Travis is deprecating the travis-ci.org website. All users should use travis-ci.com

#### CodeCov

CodeCov is a service that tells you how expansive your tests are (i.e., how much of your code is untested)

To sign up, visit the [CodeCov website](http://codecov.io/), and click “sign up.” You should see something like this

<img src="_static/figures/codecov-1.png" style="width:45%;height:45%">

  
Next, click “add a repository” and *enable private scope* (this allows CodeCov to service your private projects)

The result should be

<img src="_static/figures/codecov-2.png" style="width:45%;height:45%">

  
This is all we need for now

### Julia Setup

In [1]:
using InstantiateFromURL
activate_github("QuantEcon/QuantEconLectureAllPackages", tag = "v0.9.0") # activate the QuantEcon environment

using LinearAlgebra, Statistics, Compat # load common packaes

┌ Warning: On Windows, creating file symlinks requires Administrator privileges
└ @ Base.Filesystem file.jl:789


To recall, you can get into the pkg> mode by hitting ] in the REPL

Next, let’s create a *template* for our project

This specifies metadata like the license we’ll be using (MIT by default), the location (~/.julia/dev by default), etc.

In [2]:
using PkgTemplates
ourTemplate = Template(;user="quanteconuser", plugins = [TravisCI(), CodeCov()])

Template:
  → User: quanteconuser
  → Host: github.com
  → License: MIT (Arnav Sood 2018)
  → Package directory: ~\.julia\dev
  → Minimum Julia version: v1.0
  → SSH remote: No
  → Plugins:
    • CodeCov:
      → Config file: None
      → 3 gitignore entries: "*.jl.cov", "*.jl.*.cov", "*.jl.mem"
    • TravisCI:
      → Config file: Default
      → 0 gitignore entries

Let’s create a specific project based off this template

In [3]:
generate("ExamplePackage.jl", ourTemplate)

Generating project ExamplePackage:
    C:\Users\Arnav Sood\.julia\dev\ExamplePackage\Project.toml
    C:\Users\Arnav Sood\.julia\dev\ExamplePackage\src/ExamplePackage.jl


┌ Info: Initialized git repo at C:\Users\Arnav Sood\.julia\dev\ExamplePackage
└ @ PkgTemplates C:\Users\Arnav Sood\.julia\packages\PkgTemplates\DyEmW\src\generate.jl:22
┌ Info: Set remote origin to https://github.com/quanteconuser/ExamplePackage.jl
└ @ PkgTemplates C:\Users\Arnav Sood\.julia\packages\PkgTemplates\DyEmW\src\generate.jl:41


  Updating registry at `C:\Users\Arnav Sood\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `C:\Users\Arnav Sood\.julia\dev\ExamplePackage\Project.toml`
  [8dfed614] + Test 
  Updating `C:\Users\Arnav Sood\.julia\dev\ExamplePackage\Manifest.toml`
  [2a0f44e3] + Base64 
  [8ba89e20] + Distributed 
  [b77e0a4c] + InteractiveUtils 
  [8f399da3] + Libdl 
  [37e2e46d] + LinearAlgebra 
  [56ddb016] + Logging 
  [d6f4376e] + Markdown 
  [9a3f8284] + Random 
  [9e88b42a] + Serialization 
  [6462fe0b] + Sockets 
  [8dfed614] + Test 
  Updating registry at `C:\Users\Arnav Sood\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `C:\Users\Arnav Sood\.julia\dev\ExamplePackage\Project.toml`
 [no changes]
  Updating `C:\Users\Arnav Sood\.julia\dev\ExamplePackage\Manifest.toml`
  [2a0f44e3] - Base64 
  [8ba89e20] - Distributed 

┌ Info: Committed 9 files/directories: src/, Project.toml, Manifest.toml, test/, REQUIRE, README.md, .gitignore, LICENSE, .travis.yml
└ @ PkgTemplates C:\Users\Arnav Sood\.julia\packages\PkgTemplates\DyEmW\src\generate.jl:64


If we navigate to the package directory (shown in the output), we should see something like

<img src="_static/figures/testing-dir.png" style="width:45%;height:45%">

### Adding Project to Git

The next step is to add this project to Git version control

First, open the repository screen in your account as discussed previously. We’ll want the following settings

<img src="_static/figures/testing-git1.png" style="width:45%;height:45%">

  
In particular

- The repo you create should have the same name as the project we added  
- We should leave the boxes unchecked for the README.md, LICENSE, and .gitignore, since these are handled by PkgTemplates  


Then, drag and drop your folder from your ~/.julia/dev directory to GitHub Desktop

Click the “publish branch” button to upload your files to GitHub

If you navigate to your git repo (ours is [here](https:https://github.com/quanteconuser/ExamplePackage.jl/)), you should see something like

<img src="_static/figures/testing-git2.png" style="width:45%;height:45%">

### Adding Project to Julia Package Manager

We also want Julia’s package manager to be aware of the project

First, open a REPL in the newly created project directory, either by noting the path printed above, or by running

In [4]:
DEPOT_PATH

3-element Array{String,1}:
 "C:\\Users\\Arnav Sood\\.julia"                                          
 "C:\\Users\\Arnav Sood\\AppData\\Local\\Julia-1.0.0\\local\\share\\julia"
 "C:\\Users\\Arnav Sood\\AppData\\Local\\Julia-1.0.0\\share\\julia"       

And navigating to the first element, then the subdirectory /dev/ExamplePackage

Note the lack of .jl!

You can change the path of a Julia REPL by running

In [5]:
cd(joinpath(DEPOT_PATH[1], "dev", "ExamplePackage"))

Then, run

In [6]:
] activate 

To get into the main Julia environment, and 

In [7]:
] dev .

 Resolving package versions...
  Updating `C:\Users\Arnav Sood\.julia\environments\v1.0\Project.toml`
  [55b976a0] + ExamplePackage v0.1.0 [`C:\Users\Arnav Sood\.julia\environments\v1.0\..\..\dev\ExamplePackage`]
  Updating `C:\Users\Arnav Sood\.julia\environments\v1.0\Manifest.toml`
  [55b976a0] + ExamplePackage v0.1.0 [`C:\Users\Arnav Sood\.julia\environments\v1.0\..\..\dev\ExamplePackage`]


We see the change reflected in our default package list

In [8]:
] st

    Status `C:\Users\Arnav Sood\.julia\environments\v1.0\Project.toml`
  [c52e3926] Atom v0.7.10
  [31c24e10] Distributions v0.16.4
  [55b976a0] ExamplePackage v0.1.0 [`C:\Users\Arnav Sood\.julia\environments\v1.0\..\..\dev\ExamplePackage`]
  [2fe49d83] Expectations v1.0.2
  [7073ff75] IJulia v1.13.0
  [43edad99] InstantiateFromURL v0.1.0
  [e5e0dc1b] Juno v0.5.3
  [429524aa] Optim v0.17.1
  [d96e819e] Parameters v0.10.1
  [14b8a8f1] PkgTemplates v0.3.0
  [91a5bcdd] Plots v0.21.0


### Using the Package Manager

Now, from any Julia terminal in the future, we can run

In [9]:
] activate 

^ To simulate a fresh terminal 

In [10]:
using ExamplePackage

┌ Info: Precompiling ExamplePackage [55b976a0-e2c6-11e8-0547-a77fa1cb3ad1]
└ @ Base loading.jl:1186
┌ Warning: Package ExamplePackage does not have Expectations in its dependencies:
│ - If you have ExamplePackage checked out for development and have
│   added Expectations as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ExamplePackage
└ Loading Expectations into ExamplePackage from project dependency, future warnings for ExamplePackage are suppressed.


To use its exported functions

We can also get the path to this by running

In [11]:
using ExamplePackage
pathof(ExamplePackage) # returns path to src/ExamplePackage.jl

"C:\\Users\\Arnav Sood\\.julia\\dev\\ExamplePackage\\src\\ExamplePackage.jl"

## Project Structure

Let’s unpack the structure of the generated project

- The first directory, .git, holds the version control information  
- The src directory contains the project’s source code. Currently, it should contain only one file (ExamplePackage.jl), which reads  

```none
module ExamplePackage

greet() = print("Hello World!")

end # module
```


- Likewise, the test directory should have only one file (runtests.jl), which reads:  

```none
using ExamplePackage
using Test

@testset "ExamplePackage.jl" begin
    # Write your own tests here.
end
```


In particular, the workflow is to export objects we want to test (using ExamplePackage), and test them using Julia’s Test module

The other important text files for now are

- Project.toml and Manifest.toml, which contain dependency information  
- The .gitignore file (which may display as an untitled file), which contains files and paths for git to ignore  

## Project Workflow

### Dependency Management

#### Environments

For the following, make sure that you have an activated REPL (that is, a REPL where you’ve run pkg> activate ExamplePackage, or the original REPL where we generated the package)

This means that the ExamplePackage is our *active environment*

Any package operations we execute will be reflected in our ExamplePackage.jl directory’s TOML

Likewise, the only packages Julia knows about are those in the ExamplePackage.jl TOML. For example

In [12]:
] activate ExamplePackage

In [13]:
using QuantEcon # fails, even though QuantEcon is on the machine

ArgumentError: ArgumentError: Package QuantEcon not found in current path:
- Run `Pkg.add("QuantEcon")` to install the QuantEcon package.


This allows us to share the project with others, who can exactly reproduce the state used to build and test it

See the [Pkg3 docs](https://docs.julialang.org/en/v1/stdlib/Pkg/) for more information

#### Pkg Operations

For now, let’s just try adding a dependency

In [14]:
] add Expectations

 Resolving package versions...
  Updating `Project.toml`
  [2fe49d83] + Expectations v1.0.2
  Updating `Manifest.toml`
  [7d9fca2a] + Arpack v0.3.0
  [9e28174c] + BinDeps v0.8.10
  [b99e7846] + BinaryProvider v0.5.2
  [34da2185] + Compat v1.3.0
  [864edb3b] + DataStructures v0.14.0
  [31c24e10] + Distributions v0.16.4
  [2fe49d83] + Expectations v1.0.2
  [442a2c76] + FastGaussQuadrature v0.3.2
  [e1d29d7a] + Missings v0.3.1
  [bac558e1] + OrderedCollections v1.0.2
  [90014a1f] + PDMats v0.9.5
  [1fd47b50] + QuadGK v2.0.2
  [79098fc4] + Rmath v0.5.0
  [a2af1166] + SortingAlgorithms v0.3.1
  [276daf66] + SpecialFunctions v0.7.2
  [2913bbd2] + StatsBase v0.25.0
  [4c63d2b9] + StatsFuns v0.7.0
  [30578b45] + URIParser v0.4.0
  [2a0f44e3] + Base64 
  [ade2ca70] + Dates 
  [8bb1440f] + DelimitedFiles 
  [8ba89e20] + Distributed 
  [b77e0a4c] + InteractiveUtils 
  [76f85450] + LibGit2 
  [8f399da3] + Libdl 
  [37e2e46d] + LinearAlgebra 
  [56ddb016] + Logging 
  [d6f4376e] + Markdown 
  [a63a

Our Project.toml should now read something like:

In [ ]:
name = "ExamplePackage"
uuid = "f85830d0-e1f0-11e8-2fad-8762162ab251"
authors = ["QuantEcon User <quanteconuser@gmail.com>"]
version = "0.1.0"

[deps]
Expectations = "2fe49d83-0758-5602-8f54-1f90ad0d522b"

[extras]
Test = "8dfed614-e22c-5e08-85e1-65c5234f0b40"

[targets]
test = ["Test"]

The Manifest.toml (which tracks exact versions) has changed as well, to include a list of sub-dependencies and versions

<img src="_static/figures/testing-atom-manifest.png" style="width:45%;height:45%">

  
There are also other commands you can run from the activated environment

- pkg> up will update all dependencies to their latest versions  
- pkg> instantiate will make sure the dependencies exist on the local machine  
- pkg> rm PackageName will remove PackageName as a dependency  


To quit the active environment and return to the base (v1.0), simply run

In [15]:
] activate

Without any arguments

### Writing Code

The basic idea is to work in tests/runtests.jl, while reproducible functions should go in the src/ExamplePackage.jl

For example, let’s say we add Distributions.jl and edit the source to read as follows:

In [16]:
] activate ExamplePackage

In [17]:
] add Distributions

 Resolving package versions...
  Updating `Project.toml`
  [31c24e10] + Distributions v0.16.4
  Updating `Manifest.toml`
 [no changes]


In [ ]:
module ExamplePackage

greet() = print("Hello World!")

using Expectations, Distributions

function foo(μ = 1., σ = 2.)
    d = Normal(μ, σ)
    E = expectation(d)
    return E(x -> sin(x))
end

export foo

end # module

Let’s try calling this from a fresh Julia REPL

In [18]:
] activate

In [19]:
using ExamplePackage

In [20]:
ExamplePackage.greet()

Hello World!

In [21]:
foo() # exported, so don't need to qualify the namespace

0.11388071406436832

### Jupyter Workflow

We can also call this function from a Jupyter notebook

Let’s create a new output directory in our project, and run jupyter lab from it. Call a new notebook output.ipynb

<img src="_static/figures/testing-output.png" style="width:45%;height:45%">

  
From here, we can use our package’s functions in the usual way. This lets us produce neat output examples, without re-defining everything

We can also edit it interactively inside the notebook

<img src="_static/figures/testing-notebook.png" style="width:45%;height:45%">

  
The change will be reflected in the Project.toml file:

In [ ]:
name = "ExamplePackage"
uuid = "f85830d0-e1f0-11e8-2fad-8762162ab251"
authors = ["QuantEcon User <quanteconuser@gmail.com>"]
version = "0.1.0"

[deps]
Distributions = "31c24e10-a181-5473-b8eb-7969acd0382f"
Expectations = "2fe49d83-0758-5602-8f54-1f90ad0d522b"
Parameters = "d96e819e-fc66-5662-9728-84c9c7592b0a"

[extras]
Test = "8dfed614-e22c-5e08-85e1-65c5234f0b40"

[targets]
test = ["Test"]

And the Manifest as well

Be sure to add output/.ipynb_checkpoints to your .gitignore file, so that’s not checked in

Make sure you’ve activated the project environment (] activate ExamplePackage) before you try to propagate changes

### Collaborative Work

For someone else to get the package, they simply need to run

In [ ]:
] dev https://github.com/quanteconuser/ExamplePackage.jl.git

This will place the repository inside their ~/.julia/dev folder, and they can drag-and-drop it to GitHub desktop in the usual way

They can then collaborate as they would on other git repositories

In particular, they can run

In [ ]:
] activate ExamplePackage

In [ ]:
] instantiate

To make sure the right dependencies are installed on their machine

## Unit Testing

It’s important to make sure that your code is well-tested

There are a few different kinds of test, each with different purposes

- *Unit testing* makes sure that individual pieces of a project function as expected  
- *Integration testing* makes sure that they work together as expected  
- *Regression testing* makes sure that behavior is unchanged over time  


In this lecture, we’ll focus on unit testing

### The Test Module

Julia provides testing features through a built-in package called Test, which we get by using Test

The basic object is the macro @test

In [22]:
using Test
@test 1 == 1
@test 1 ≈ 1

Test Passed

Tests will pass if the condition is true, or fail otherwise

If a test is failing, we should *flag it and move on*

In [23]:
@test_broken 1 == 2

Test Broken
  Expression: 1 == 2

This way, we still have access to information about the test, instead of just deleting it or commenting it out

There are other test macros, that check for things like error handling and type-stability. Advanced users can check the [Julia docs](https://docs.julialang.org/en/v1/stdlib/Test/)

### Example

Let’s add some unit tests for the foo() function we defined earlier. Our tests/runtests.jl file should look like this

In [24]:
using ExamplePackage
using Test

@test foo() == 0.11388071406436832
@test foo(1, 1.5) == 0.2731856314283442
@test_broken foo(1, 0) # tells us this is broken

Test Broken
  Expression: foo(1, 0)

And run it by running

In [ ]:
] test

from an activated REPL

### Test Sets

By default, the runtests.jl folder starts off with a @testset

This is useful for organizing different batches of tests, but for now we can simply ignore it

To learn more about test sets, see [the docs](https://docs.julialang.org/en/v1/stdlib/Test/index.html#Working-with-Test-Sets-1/)

### Running Tests Locally

There are a few different ways to run the tests for your package

- From a fresh REPL, run pkg> test ExamplePackage  
- From an activated REPL, simply run pkg> test (recall that you can activate with pkg> activate ExamplePackage)  
- Hit shift-enter in Atom on the actual runtests.jl file (as below). Recall that we can get the path of the package by running using ExamplePackage; pathof(ExamplePackage) from an unactivated REPL  

## Continuous Integration with Travis

### Setup

By default, Travis should have access to all your repositories and deploy automatically

This includes private repos if you’re on a student developer pack or an academic plan (Travis detects this automatically)

To change this, go to “settings” under your GitHub profile

<img src="_static/figures/git-settings.png" style="width:45%;height:45%">
  
Click “Applications,” then “Travis CI,” then “Configure,” and choose the repos you want to be tracked

### Build Options

By default, Travis will run builds for new commits and PRs for every tracked repo with a .travis.yml file

We can see ours by opening it in Atom

In [ ]:
# Documentation: http://docs.travis-ci.com/user/languages/julia/
language: julia
os:
- linux
- osx
julia:
- 1.0
- nightly
matrix:
allow_failures:
    - julia: nightly
fast_finish: true
notifications:
email: false
after_success:
- julia -e 'using Pkg; Pkg.add("Coverage"); using Coverage; Codecov.submit(process_folder())'

This is telling Travis to build the project in Julia, on OSX and Linux, using Julia v1.0 and the latest (“nightly”)

It also says that if the nightly version doesn’t work, that shouldn’t register as a failure

**Note** You won’t need OSX unless you’re building something Mac-specific, like iOS or Swift. You can delete those lines to speed up the build. Likewise for nightly Julia

### Working with Builds

As above, builds are triggered whenever we push changes or open a pull request

For example, if we push our changes to the server and then click the Travis badge on the README, we should see something like

<img src="_static/figures/travis-progress.png" style="width:45%;height:45%">

  
This gives us an overview of all the builds running for that commit

To inspect a build more closely (say, if it fails), we can click on it and expand the log options

<img src="_static/figures/travis-log.png" style="width:45%;height:45%">

  
Note that the build times here aren’t informative, because we can’t generally control the hardware to which our job is allocated

We can also cancel specific jobs, either from their specific pages or by clicking the grey “x” button on the dashboard

Lastly, we can trigger builds manually (without a new commit or PR) from the Travis overview

<img src="_static/figures/travis-trigger.png" style="width:45%;height:45%">

  
To commit *without* triggering a build, simply add [ci skip] somewhere inside the commit message

### Travis and Pull Requests

One key feature of Travis is the ability to see at-a-glance whether PRs pass tests before merging them

This happens automatically when Travis is enabled on a repository

For an example of this feature, see [this PR](https://github.com/QuantEcon/Games.jl/pull/65/) in the Games.jl repository

## CodeCoverage

Beyond the success or failure of our test suite, we also want to know how much of our code the tests cover

The tool we use to do this is called [CodeCov](http://codecov.io)

### Setup

You’ll find that codecov is automatically enabled for public repos with Travis

For private ones, you’ll need to first get an access token

Add private scope in the CodeCov website, just like we did for Travis

Navigate to the repo settings page (i.e., https://codecov.io/gh/quanteconuser/ExamplePackage.jl/settings for our repo) and copy the token

Next, go to your travis settings and add an environment variable as below

<img src="_static/figures/travis-settings.png" style="width:45%;height:45%">

### Interpreting Results

Click the CodeCov badge to see the build page for your project

This shows us that our tests cover 50 % of our functions in src//

To get a more granular view, we can click the src// and the resultant filename

<img src="_static/figures/codecov.png" style="width:45%;height:45%">

  
This shows us precisely which methods (and parts of methods) are untested

## Benchmarking

Another goal of testing is to make sure that code doesn’t slow down significantly from one version to the next

We can do this using tools provided by the BenchmarkTools.jl package

See the need for speed lecture for more details

## Additional Notes

- The [JuliaCI](https://github.com/JuliaCI/) organization provides more Julia utilities for continuous integration and testing  
- This [Salesforce document](https://developer.salesforce.com/page/How_to_Write_Good_Unit_Tests/) has some good lessons about writing and testing code  